In [74]:
import pandas as pd 
import numpy as np
import xml.etree.ElementTree as ET

periode=3 #-------------------------------

tree = ET.parse(f'XMLs\\periode_{periode-1}.xml')
root = tree.getroot()

base_df=pd.read_csv('Data_pl\\base_table_01.csv',index_col='Unnamed: 0')
base_df['Lieferfrist']=base_df['Lieferfrist'].str.replace(',','.').astype('float')
base_df['Abweichung']=base_df['Abweichung'].str.replace(',','.').astype('float')

dfl=[]
for a in root.find('warehousestock').findall('article'):
    dfl.append(a.attrib)
bestands_df = pd.DataFrame(dfl) 

for col in bestands_df.columns:
    try:
        bestands_df[col]=bestands_df[col].astype(int)
    except:
        bestands_df[col]=bestands_df[col].str.replace(',','.')
        bestands_df[col]=bestands_df[col].astype(float)

dfl=[]
for a in root.find('futureinwardstockmovement').findall('order'):
    dfl.append(a.attrib)
futur_orders_df = pd.DataFrame(dfl)

for col in futur_orders_df.columns:
    futur_orders_df[col]=futur_orders_df[col].astype(int)

dfl=[]
for a in root.find('inwardstockmovement').findall('order'):
    dfl.append(a.attrib)
orders_in_df = pd.DataFrame(dfl)

for col in orders_in_df.columns:
    try:
        orders_in_df[col]=orders_in_df[col].astype(int)
    except:
        orders_in_df[col]=orders_in_df[col].str.replace(',','.')
        orders_in_df[col]=orders_in_df[col].astype(float)

In [75]:
periodes=list(range(periode,periode+4))

plan=[#--------------------------------------------------------
['P1',200,150,200,250],
['P2',100,100,100,150],
['P3',50,50,50,50]]

plan=pd.DataFrame(plan,columns=['Pr']+periodes)
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)
plan

,3,4,5,6
Pr,,,,
P1,150,150,200,250
P2,100,100,100,150
P3,50,50,50,50


In [76]:
merge_1=pd.merge(base_df,bestands_df,left_on='NR. K',right_on='id').drop(columns=['id','startamount','pct','price','stockvalue'])

for col in periodes:
    merge_1[f'Bedarf_{col}']=merge_1[['P1','P2','P3']]@plan[col]

In [77]:
merge_2=pd.merge(futur_orders_df,base_df,left_on='article',right_on='NR. K')
merge_2['avg_order_time']=merge_2['Lieferfrist']+merge_2['Abweichung']
merge_2=merge_2.drop(columns=['id','NR. K','P1','P2','P3','Diskontmenge','Lieferfrist','Abweichung'])

merge_2['avg_order_time']=merge_2.apply(lambda x : x['avg_order_time'] if x['mode'] == 5 else x['avg_order_time']/2,axis=1)
merge_2['expected_arival_periode']=merge_2['orderperiod']+merge_2['avg_order_time']
merge_2['expected_arival_periode']=np.floor(merge_2['expected_arival_periode'])

In [78]:
# current_col=5
def order_sum_at_periode(x):
    global current_col
    return sum(merge_2[(merge_2['article']==x['NR. K'])&(merge_2['expected_arival_periode']==current_col)]['amount'])

In [79]:
amount_df=merge_1.copy()
current_col=periodes[0]

amount_df[(f'paa_{periodes[0]}','start')]=amount_df['amount']
amount_df[(f'paa_{periodes[0]}','bed')]=amount_df[f'Bedarf_{periodes[0]}']
amount_df[(f'paa_{periodes[0]}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1)

current_col=periodes[0]-1
amount_df[(f'paa_{periodes[0]}','o_in')]+=amount_df.apply(order_sum_at_periode,axis=1)

amount_df[(f'paa_{periodes[0]}','end')]=(amount_df['amount']-amount_df[f'Bedarf_{periodes[0]}']).apply(lambda x : 0 if x<=0 else x)
amount_df[(f'paa_{periodes[0]}','end')]+=amount_df[(f'paa_{periodes[0]}','o_in')]


for col in periodes[1:]:
    current_col=col
    amount_df[(f'paa_{col}','start')]=amount_df[(f'paa_{col-1}','end')]
    amount_df[(f'paa_{col}','bed')]=amount_df[f'Bedarf_{col}']
    amount_df[(f'paa_{col}','o_in')]=amount_df.apply(order_sum_at_periode,axis=1) 
    amount_df[(f'paa_{col}','end')]=(amount_df[(f'paa_{col-1}','end')]-amount_df[f'Bedarf_{col}']).apply(lambda x : 0 if x<=0 else x)
    amount_df[(f'paa_{col}','end')]+=amount_df[(f'paa_{col}','o_in')]


In [80]:
l=[]
for i,x in enumerate(amount_df.columns):
    if type(x)!=tuple:
        l.append((x,''))
    else:
        l.append(x)

amount_df.columns=l
amount_df.columns=pd.MultiIndex.from_tuples(amount_df.columns)

In [81]:
out=amount_df[[f'paa_{x}' for x in periodes]]
out.set_index(amount_df['NR. K']).to_csv(f'Data_pl\\amount_periode_{periode}.csv')

In [82]:
def needed_order_period(x):
    for i,y in enumerate(x):
        if y ==0:
            periode=iter_out.columns[i][0].split('_')[1]
            return int(periode)
    return max(periodes)+1
    

In [83]:
l=[]
for x in out.columns:
    if (x[1]=='o_in')|(x[1]=='start'):
        l.append(x)
iter_out=out.drop(columns=l)

# needed_order_at_start_of_period
amount_df['need_order']=iter_out.apply(needed_order_period,axis=1)+1
# needed_to_order_at_start_of_period
amount_df['need_to_order']=np.floor(amount_df['need_order']-amount_df['Lieferfrist']-amount_df['Abweichung'])
amount_df['need_to_order_iff']=np.floor(amount_df['need_order']-(amount_df['Lieferfrist']+amount_df['Abweichung'])/2)

In [84]:
def min_order_amount(x):
    need_order=int( x['need_to_order'][0] )
    
    if need_order==periode:
        return 'n', x[f'Bedarf_{periode}'][0]
    
    elif need_order<periode:
        return 'f' , x[f'Bedarf_{periode}'][0]
    
    else:
        return None , 0 

In [85]:
amount_df['order_type'],amount_df['min_order_amount']=zip( *amount_df.apply(min_order_amount,axis=1) )

amount_df.to_csv(f'Order_history\\order_amount_{periode}.csv')

In [86]:
amount_df.drop(columns=[
            (                'NR. K',      ''),
            (                'P1',      ''),
            (                'P2',      ''),
            (                'P3',      ''),
            #(      'Diskontmenge',      ''),
            (            'amount',      ''),
            (          'Bedarf_3',      ''),
            (          'Bedarf_4',      ''),
            (          'Bedarf_5',      ''),
            (          'Bedarf_6',      ''),])

Lieferfrist Abweichung Diskontmenge  paa_3                     paa_4  \
                                        start    bed o_in    end  start   
0          1.8        0.4          300     50    150  300    300    300   
1          1.7        0.4          300    100    100  300    300    300   
2          1.2        0.2          300    100     50    0     50     50   
3          3.2        0.3         6100   7650   2100    0   5550   5550   
4          0.9        0.2         3600   4600   1200    0   3400   3400   
5          0.9        0.2         1800   2300    600    0   1700   1700   
6          1.7        0.4         4500   5800   1400    0   4400   4400   
7          2.1        0.5         2700   3490    900    0   2590   2590   
8          1.9        0.5          900    500    100    0    400    400   
9          1.6        0.3        22000   7600   3600    0   4000   4000   
10         2.2        0.4         3600   4620   1200    0   3420   3420   
11         1.2        0.1          900   1160    300    0    860    860   
12         1.5        0.3          900    250    300  900    900    900   
13         1.7        0.4          300    650    300    0    350    350   
14         1.5        0.3         1800   1400    600    0    800    800   
15         1.7        0.2          900   1150    300    0    850    850   
16         0.9        0.2          900   1150    300    0    850    850   
17         1.2        0.3         1800   2300    600    0   1700   1700   
18         2.0        0.5         2700   1250    300    0    950    950   
19         1.0        0.2          900   2010    900    0   1110   1110   
20         1.7        0.3          900   1150    300    0    850    850   
21         0.9        0.3          900   1150    300    0    850    850   
22         1.1        0.1          900   1240    300    0    940    940   
23         1.0        0.2         1800   2360    600    0   1760   1760   
24         1.6        0.4          600    700    300    0    400    400   
25         1.6        0.2        22000  26000  10800    0  15200  15200   
26         1.7        0.3          600    800    200    0    600    600   
27         1.6        0.5        22000  29600   7300    0  22300  22300   
28         0.7        0.2         1800   2300    600    0   1700   1700   

                ... paa_5 paa_6                  need_order need_to_order  \
      bed o_in  ...   end start    bed o_in  end                            
0     150    0  ...     0     0    250    0    0          6           3.0   
1     100    0  ...   100   100    150    0    0          7           4.0   
2      50    0  ...     0     0     50    0    0          5           3.0   
3    2100    0  ...  1000  1000   3150    0    0          7           3.0   
4    1200    0  ...   800   800   1800    0    0          7           5.0   
5     600    0  ...   400   400    900    0    0          7           5.0   
6    1400    0  ...  1400  1400   2050    0    0          7           4.0   
7     900    0  ...   640   640   1350    0    0          7           4.0   
8     100    0  ...   200   200    100    0  100          8           5.0   
9    3600    0  ...     0     0   3600    0    0          6           4.0   
10   1200    0  ...   820   820   1800    0    0          7           4.0   
11    300    0  ...   210   210    450    0    0          7           5.0   
12    300    0  ...   250   250    450    0    0          7           5.0   
13    300    0  ...     0     0    450    0    0          6           3.0   
14    600    0  ...     0     0    900    0    0          6           4.0   
15    300    0  ...   200   200    450    0    0          7           5.0   
16    300    0  ...   200   200    450    0    0          7           5.0   
17    600    0  ...   400   400    900    0    0          7           5.0   
18    300    0  ...   300   300    450    0    0          7           4.0   
19    900    0  ...     0     0   1350    0    0          6 